# Keras + Flask for REST api

In [1]:
import pickle
import numpy as np
import flask
import io
from keras.models import Model
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
# initialize our Flask application and the Keras model
app = flask.Flask(__name__)
model = None
english_tokenizer = None
french_tokenizer = None
y_id_to_word = {}

## Loading our model and tokenizer

In [3]:
def load_translation_model():
    # load the pre-trained Keras model 
    global model
    global english_tokenizer
    global french_tokenizer
    global y_id_to_word
    model = load_model('translation.h5')
    # loading
    with open('english_tokenizer.pickle', 'rb') as handle:
        english_tokenizer = pickle.load(handle)
    with open('french_tokenizer.pickle', 'rb') as handle:
        french_tokenizer = pickle.load(handle)
    y_id_to_word = {value: key for key, value in french_tokenizer.word_index.items()}
    y_id_to_word[0] = '<PAD>'

## Preprocessing the text for input to the model

In [4]:
def prepare_text(sentence):
    sentence = [english_tokenizer.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=15, padding='post')
    sentence = np.array(sentence)
    return sentence

## Converting numbers back to words

In [5]:
def logits_to_text(logits, tokenizer = french_tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = y_id_to_word

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [6]:
@app.route("/predict")
def predict():
    data = {"success": False}
    sentence = flask.request.args.get('sentence')
    print(sentence)
    sentence = prepare_text(sentence)
    sentence = model.predict(sentence,1)
    sentence = logits_to_text(sentence)
    data["success"] = True
    data["prediction"] = sentence
    # return the data dictionary as a JSON response
    return flask.jsonify(data)
    

In [ ]:
# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
        "please wait until server has fully started"))
    load_translation_model()
    app.run()

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2018-10-20 19:27:05,267] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/avinsharma/miniconda3/envs/keras/lib/python

banana


In [ ]:
print(english_tokenizer.word_index)